# Test Database Formation
This notebook turns an expedition folder with the results of multiple sequencings and a description excel file into a small database for testing the difference consensus/identification pipelines. The unsorted expedition folder is added in the `input` folder and the resulting test database is saved in `testdbs`.

## Format of the input expedition folder

The expedition folder does not need to be sorted beforehand to contain only the fastq files. However, it must respect some conventions:
- all the fastq files should be in folders named `barcode{barcode#}`. These folders should either be found in a `fastq_pass` parent directory or a `barcoding` parent directory, as is usually the case for expedition folders.
- the `barcode{barcode#}` folders should only contain fastq files or .gz zips of fastq files. The presence of other types of files will cause the code to crash
- the expedition folder should contain an excel describing the samples called `Description_sample.xlsx`

## Format of the description file

The description excel file has a very specific format and should be filled in from the template provided. You can check [this example](input/matK_rbcL_trnh_ITS_12samples_publicationsummer2022_9Qiagen_3MN/Description_sample.xlsx) on how to fill it in.
- the first sheet contains general information about the expedition (columns: ref, experiment, description, notes)
- the second sheet contains the barcode sequences
- the third sheet contains the samples and genes sequenced for each barcode (columns: Barcode, Species, gene, Volume(ml), note)
- the fourth sheet contains the primers used

#### Imports

In [8]:
from general_helpers import *
import os.path as ospath
import pandas as pd
import os

#### Setting parameters for expedition folder conversion

all of the databases are stored in the folder `testdbs`. Please fill in:
- the name of the expedition folder you want to convert in `input_expedition_folder` 
- the name you want to give to the newly created database in `name_of_db`
- the `permissive_search` parameter (True or False). This has to do with the modality of searching for reference sequences in GenBank. For each species and gene sequenced, the newly sorted test database will contain the reference fasta sequences it extracts from GenBank. By default, the code will try an advanced search to get accurate results (ex: "Ailanthus altissima[species] AND matK[gene name] AND 0:5000[sequence_length]" ). If this advanced search fails, the permissive search parameter determines whether the code should:
    - `False`: not include a reference sequence 
    - `True`: try to find a reference sequence through a "permissive" less accurate search (True) (ex: "Ailanthus altissima matK")

While the advanced GenBank search returns very accurate results when it works, it very often does not return anything, which is why we recommend setting this parameter to `True` while taking the reference sequences with a grain of salt.

In [9]:
input_expedition_folder = "matK_rbcL_trnh_ITS_27samples_Jardin_botanique_1st"
name_of_db = "expedition_jardin_botanique"
permissive_search = True

#### Finding the location of the description sample excel file

In [10]:
db_path = ospath.join("input", input_expedition_folder)
main_dir = None
for root, dirs, files in os.walk(db_path):
    if "Description_sample.xlsx" in files:
        main_dir = root
        description_path = ospath.join(main_dir,"Description_sample.xlsx")
    break
if main_dir == None:
    print("There is no excel description folder for expedition")
else: 
    print(description_path)

input/matK_rbcL_trnh_ITS_27samples_Jardin_botanique_1st/Description_sample.xlsx


#### extracting information about genes sequenced for each plant, barcode sequences and general expedition information.

In [11]:
info_db = pd.read_excel(description_path,sheet_name=0,index_col=0)
barseq_db = pd.read_excel(description_path,sheet_name=1)
sample_db = pd.read_excel(description_path,sheet_name=2)
primer_db = pd.read_excel(description_path,sheet_name=3)
print("Experiment:", info_db.experiment[0])
sample_db.note[sample_db.note.isna()] = "No Note"
print(sample_db.head())

Experiment: Sequencing of MN samples with purification
   Barcode                   Species       gene  Volume(ml)     note
0       68             Iris sibirica  matk, ITS         NaN  No Note
1       69      Eleocharis Palustris        ITS         NaN  No Note
2       70  Cercidyphyllum japonicum        ITS         NaN  No Note
3       71         Acer opalus Mill.  trnH-psbA         NaN  No Note
4       72         Acer opalus Mill.        ITS         NaN  No Note


/tmp/ipykernel_49875/1788768446.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_db.note[sample_db.note.isna()] = "No Note"


#### Creating the new test database
for each sample, the corresponding fastq pass reads are extracted in one file. Thre is also a fasta file with the reference sequences for the genes sequenced from GenBank.

In [12]:
#creating the new database folder
new_db = ospath.join("testdbs", name_of_db)
if not ospath.exists(new_db):
    os.makedirs(new_db)

#iterating over the samples
for index, row in sample_db.iterrows():

    #new folder for each sample
    species = row["Species"].replace(" ", "_")

    genes = row["gene"].split(",")
    genes =[gene.replace(" ", "") for gene in genes]

    gene_str= row["gene"].replace(" ", "")
    gene_str= gene_str.replace(",","_")
    
    new_dir= ospath.join(new_db, species+"_"+ gene_str+ "_barcode"+str(row["Barcode"]))
    if not ospath.exists(new_dir):
        os.makedirs(new_dir)

    #extracting the fastq from the input expedition folder
    file_location = ospath.join(new_dir, species+ gene_str+ "_barcode"+str(row["Barcode"]) )
    extract_fastq(main_dir, row["Barcode"], file_location)

    #downloading the reference sequences from NCBI
    reference_seq_location = ospath.join(new_dir, species+"_reference_seq.fasta")
    for gene in genes:
        download_sequence(row["Species"], gene, reference_seq_location, 0, 5000, permissive_search=permissive_search)



input/matK_rbcL_trnh_ITS_27samples_Jardin_botanique_1st/jardin_botanique/barcoding/barcode68
input/matK_rbcL_trnh_ITS_27samples_Jardin_botanique_1st/jardin_botanique/20230720_1819_MN41570_ANQ493_85a742ea/fastq_pass/barcode68


input/matK_rbcL_trnh_ITS_27samples_Jardin_botanique_1st/jardin_botanique/20230720_1819_MN41570_ANQ493_85a742ea/fastq_pass/barcode69
input/matK_rbcL_trnh_ITS_27samples_Jardin_botanique_1st/jardin_botanique/20230720_1819_MN41570_ANQ493_85a742ea/fastq_pass/barcode70
input/matK_rbcL_trnh_ITS_27samples_Jardin_botanique_1st/jardin_botanique/20230720_1819_MN41570_ANQ493_85a742ea/fastq_pass/barcode72
input/matK_rbcL_trnh_ITS_27samples_Jardin_botanique_1st/jardin_botanique/barcoding/barcode73
input/matK_rbcL_trnh_ITS_27samples_Jardin_botanique_1st/jardin_botanique/20230720_1819_MN41570_ANQ493_85a742ea/fastq_pass/barcode73
input/matK_rbcL_trnh_ITS_27samples_Jardin_botanique_1st/jardin_botanique/barcoding/barcode74
input/matK_rbcL_trnh_ITS_27samples_Jardin_botanique_1st/jardin_botanique/20230720_1819_MN41570_ANQ493_85a742ea/fastq_pass/barcode74
input/matK_rbcL_trnh_ITS_27samples_Jardin_botanique_1st/jardin_botanique/20230720_1819_MN41570_ANQ493_85a742ea/fastq_pass/barcode75
input/matK_rbcL_trnh_I

#### Logging information about the expedition as csv files to be easily reimported as Pandas DataFrames.

In [ ]:
info_db.to_csv(ospath.join(new_db, "general_info.csv"))
sample_db.to_csv(ospath.join(new_db, "sample_info.csv"))
primer_db.to_csv(ospath.join(new_db, "primer_info.csv"))